In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['RA']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('rheumatoid arthritis', 8999),
 ('retinoic acid', 7223),
 ('rosmarinic acid', 317),
 ('right atrial', 81),
 ('right atrium', 77),
 ('ras associated', 65),
 ('radial artery', 55),
 ('robust nucleus of the arcopallium', 43),
 ('room air', 38),
 ('rapidly adapting', 30),
 ('ra', 26),
 ('rotational atherectomy', 25),
 ('radix astragali', 19),
 ('refractory anemia', 18),
 ('raddeanin a', 18),
 ('rectus abdominis', 15),
 ('radiation attenuated', 14),
 ('retrograde amnesia', 13),
 ('related abundance', 13),
 ('riemerella anatipestifer', 13),
 ('region anesthesia', 13),
 ('rheumatoid', 13),
 ('renin angiotensin', 12),
 ('retinyl acetate', 12),
 ('rheumatic arthritis', 10),
 ('rapidarc', 9),
 ('rheumatoid arthritic', 8),
 ('rats aorta', 8),
 ('radicicol', 7),
 ('receptor antagonist', 7),
 ('rapamycin', 7),
 ('region anaesthesia', 7),
 ('risk assess', 6),
 ('recovered activity', 6),
 ('renal artery', 6),
 ('ras associated domain', 6),
 ('rassf1a specific sirna', 6),
 ('reperfusion arrhythmias'

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'MESH:D001172': 'Arthritis, Rheumatoid',
 'CHEBI:CHEBI:26536': 'retinoic acid',
 'CHEBI:CHEBI:17226': 'rosmarinic acid',
 'MESH:D017534': 'Radial Artery',
 'MESH:D000388': 'Air',
 'MESH:D017073': 'Atherectomy',
 'MESH:C027492': 'Huang Qi',
 'MESH:D000753': 'Anemia, Refractory',
 'MESH:C584473': 'raddeanin A',
 'MESH:D017568': 'Rectus Abdominis',
 'MESH:D000648': 'Amnesia, Retrograde',
 'MESH:C000639115': 'Riemerella anatipestifer',
 'CHEBI:CHEBI:32095': 'retinyl acetate',
 'MESH:D012213': 'Rheumatic Fever',
 'MESH:D001011': 'Aorta',
 'CHEBI:CHEBI:556075': 'radicicol',
 'CHEBI:CHEBI:9168': 'sirolimus',
 'MESH:D012077': 'Renal Artery',
 'MESH:D034741': 'RNA, Small Interfering',
 'MESH:D001145': 'Arrhythmias, Cardiac',
 'MESH:D010361': 'Patients',
 'HP:HP:0000104': 'Renal agenesis',
 'MESH:D001251': 'Astigmatism',
 'MESH:D001108': 'Architecture',
 'CHEBI:CHEBI:34922': 'picloram',
 'CHEBI:CHEBI:8891': 'Roridin A',
 'HP:HP:0012834': 'Right',
 'MESH:D000042': 'Absorption',
 'EFO:0002752': '

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 03:51:35] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'and right': 'ungrounded',
  'arg ala': 'CHEBI:CHEBI:73810',
  'artery resistance': 'ungrounded',
  'ra': 'ungrounded',
  'ra patients': 'ungrounded',
  'ra treated': 'ungrounded',
  'raddeanin a': 'MESH:C584473',
  'radial access': 'ungrounded',
  'radial artery': 'ungrounded',
  'radiation attenuated': 'ungrounded',
  'radicicol': 'CHEBI:CHEBI:556075',
  'radix astragali': 'ungrounded',
  'random access': 'ungrounded',
  'rap associated': 'ungrounded',
  'rapamycin': 'CHEBI:CHEBI:9168',
  'rapidarc': 'ungrounded',
  'rapidly act human insulin analogs': 'ungrounded',
  'rapidly adapting': 'ungrounded',
  'ras associated': 'IP:IPR000159',
  'ras associated domain': 'ungrounded',
  'rassf1a specific sirna': 'ungrounded',
  'rats aorta': 'ungrounded',
  'rats thoracic aorta': 'MESH:D001013',
  'reactive aggressive': 'ungrounded',
  'receptor antagonist': 'ungrounded',
  'recombinant antigen': 'ungrounded',
  'reconstruction algorithm': 'ungrounded',
  'recovered activity': 'ungrounded'

In [11]:
grounding_map, names, pos_labels = [{'and right': 'ungrounded',
  'arg ala': 'CHEBI:CHEBI:73810',
  'artery resistance': 'ungrounded',
  'ra': 'ungrounded',
  'ra patients': 'ungrounded',
  'ra treated': 'ungrounded',
  'raddeanin a': 'MESH:C584473',
  'radial access': 'ungrounded',
  'radial artery': 'ungrounded',
  'radiation attenuated': 'ungrounded',
  'radicicol': 'CHEBI:CHEBI:556075',
  'radix astragali': 'ungrounded',
  'random access': 'ungrounded',
  'rap associated': 'ungrounded',
  'rapamycin': 'CHEBI:CHEBI:9168',
  'rapidarc': 'ungrounded',
  'rapidly act human insulin analogs': 'ungrounded',
  'rapidly adapting': 'ungrounded',
  'ras associated': 'IP:IPR000159',
  'ras associated domain': 'ungrounded',
  'rassf1a specific sirna': 'ungrounded',
  'rats aorta': 'ungrounded',
  'rats thoracic aorta': 'MESH:D001013',
  'reactive aggressive': 'ungrounded',
  'receptor antagonist': 'ungrounded',
  'recombinant antigen': 'ungrounded',
  'reconstruction algorithm': 'ungrounded',
  'recovered activity': 'ungrounded',
  'rectus abdominis': 'ungrounded',
  'recycled aggregation': 'ungrounded',
  'redox adapting': 'ungrounded',
  'reduced and alkylated': 'ungrounded',
  'refractive astigmatism': 'MESH:D001251',
  'refractory anemia': 'MESH:D000753',
  'region anaesthesia': 'ungrounded',
  'region anesthesia': 'ungrounded',
  'related absorption': 'ungrounded',
  'related abundance': 'ungrounded',
  'related affinity': 'ungrounded',
  'related amplitude': 'ungrounded',
  'renal agenesis': 'HP:HP:0000104',
  'renal artery': 'MESH:D012077',
  'renin angiotensin': 'ungrounded',
  'reperfusion arrhythmias': 'MESH:D001145',
  'repolarization alternans': 'ungrounded',
  'resistance artery': 'ungrounded',
  'resource allocation': 'MESH:D040841',
  'respiratory arrest': 'HP:HP:0005943',
  'restricted alleviation': 'ungrounded',
  'retinoic acid': 'CHEBI:CHEBI:26536',
  'retinyl acetate': 'CHEBI:CHEBI:32095',
  'retrograde amnesia': 'MESH:D000648',
  'rheumatic arthritis': 'MESH:D001172',
  'rheumatoid': 'ungrounded',
  'rheumatoid arthritic': 'MESH:D001172',
  'rheumatoid arthritis': 'MESH:D001172',
  'rheumatoid arthritis patients': 'MESH:D001172',
  'riemerella anatipestifer': 'MESH:C000639115',
  'right angle': 'ungrounded',
  'right atria': 'ungrounded',
  'right atrial': 'ungrounded',
  'right atrium': 'ungrounded',
  'right auricle': 'ungrounded',
  'risk area': 'ungrounded',
  'risk assess': 'ungrounded',
  'robust nucleus of the archistriatum': 'MESH:D000679',
  'robust nucleus of the arcopallium': 'ungrounded',
  'robustus archistriatalis': 'ungrounded',
  'role ambiguity': 'ungrounded',
  'room air': 'ungrounded',
  'root angulation': 'ungrounded',
  'root architecture': 'ungrounded',
  'root avulsion': 'ungrounded',
  'roridin a': 'CHEBI:CHEBI:8891',
  'rosmarinic acid': 'CHEBI:CHEBI:17226',
  'rotational atherectomy': 'ungrounded',
  'rumex acetosa': 'ungrounded'},
 {'CHEBI:CHEBI:73810': 'Arg-Ala',
  'MESH:C584473': 'raddeanin A',
  'CHEBI:CHEBI:556075': 'radicicol',
  'CHEBI:CHEBI:9168': 'sirolimus',
  'IP:IPR000159': 'Ras-associating (RA) domain',
  'MESH:D001013': 'Aorta, Thoracic',
  'MESH:D001251': 'Astigmatism',
  'MESH:D000753': 'Anemia, Refractory',
  'HP:HP:0000104': 'Renal agenesis',
  'MESH:D012077': 'Renal Artery',
  'MESH:D001145': 'Arrhythmias, Cardiac',
  'MESH:D040841': 'Resource Allocation',
  'HP:HP:0005943': 'Respiratory arrest',
  'CHEBI:CHEBI:26536': 'retinoic acid',
  'CHEBI:CHEBI:32095': 'retinyl acetate',
  'MESH:D000648': 'Amnesia, Retrograde',
  'MESH:D001172': 'Arthritis, Rheumatoid',
  'MESH:C000639115': 'Riemerella anatipestifer',
  'MESH:D000679': 'Amygdala',
  'CHEBI:CHEBI:8891': 'Roridin A',
  'CHEBI:CHEBI:17226': 'rosmarinic acid'},
 ['CHEBI:CHEBI:17226', 'CHEBI:CHEBI:26536', 'IP:IPR000159', 'MESH:D001172']]

In [12]:
excluded_longforms = ['ra', 'ra patients', 'ra treated']

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [17]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [18]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [19]:
intersection1

[]

In [20]:
intersection2

[]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [21]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 04:01:47] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 04:08:45] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9768646524355177 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [22]:
classifier.stats

{'label_distribution': {'MESH:D001172': 5860,
  'CHEBI:CHEBI:26536': 5505,
  'CHEBI:CHEBI:32095': 10,
  'ungrounded': 433,
  'MESH:D001145': 4,
  'MESH:D000753': 16,
  'MESH:D012077': 5,
  'CHEBI:CHEBI:17226': 229,
  'MESH:D000648': 11,
  'HP:HP:0005943': 3,
  'MESH:D000679': 3,
  'IP:IPR000159': 45,
  'CHEBI:CHEBI:8891': 4,
  'CHEBI:CHEBI:556075': 6,
  'MESH:C000639115': 11,
  'MESH:D001013': 2,
  'HP:HP:0000104': 4,
  'CHEBI:CHEBI:9168': 6,
  'MESH:C584473': 15,
  'MESH:D040841': 1,
  'MESH:D001251': 2,
  'CHEBI:CHEBI:73810': 1},
 'f1': {'mean': 0.976865, 'std': 0.001843},
 'precision': {'mean': 0.971642, 'std': 0.00167},
 'recall': {'mean': 0.982473, 'std': 0.001984},
 'IP:IPR000159': {'f1': {'mean': 0.808421, 'std': 0.057303},
  'pr': {'mean': 0.822222, 'std': 0.133333},
  'rc': {'mean': 0.811948, 'std': 0.047826}},
 'HP:HP:0000104': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}},
 'MESH:D012077': {'f1': {'mean': 0.0, 'std': 

In [23]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [24]:
disamb.dump(model_name, results_path)

In [25]:
print(disamb.info())

Disambiguation model for RA

Produces the disambiguations:
	Amnesia, Retrograde	MESH:D000648
	Amygdala	MESH:D000679
	Anemia, Refractory	MESH:D000753
	Aorta, Thoracic	MESH:D001013
	Arg-Ala	CHEBI:CHEBI:73810
	Arrhythmias, Cardiac	MESH:D001145
	Arthritis, Rheumatoid*	MESH:D001172
	Astigmatism	MESH:D001251
	Ras-associating (RA) domain*	IP:IPR000159
	Renal Artery	MESH:D012077
	Renal agenesis	HP:HP:0000104
	Resource Allocation	MESH:D040841
	Respiratory arrest	HP:HP:0005943
	Riemerella anatipestifer	MESH:C000639115
	Roridin A	CHEBI:CHEBI:8891
	raddeanin A	MESH:C584473
	radicicol	CHEBI:CHEBI:556075
	retinoic acid*	CHEBI:CHEBI:26536
	retinyl acetate	CHEBI:CHEBI:32095
	rosmarinic acid*	CHEBI:CHEBI:17226
	sirolimus	CHEBI:CHEBI:9168

Class level metrics:
--------------------
Grounding                  	Count	F1     
      Arthritis, Rheumatoid*	5860	0.98462
              retinoic acid*	5505	0.97743
                 Ungrounded	 433	0.80556
            rosmarinic acid*	 229	0.79806
Ras-associating (

In [26]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1